# SYNTHETIC DATASET

With this notebook we will be able to create a synthetic dataset that will be used for training, validating and testing our model.

The notebook will allow the customization of input parameters that will be reflected in the generated dataset.

Output will be saved by default under the 'dataset' folder, conflicting files will be overriden.

## Desired Output

Following the work done by 'benedekrozemberczki' which can be found in the [following repository](https://github.com/benedekrozemberczki/SimGNN), we decided to adopt the same JSON structure when saving our dataset points (examples). Thus we create an helper class which will be dumped to a JSON file at time.

NOTE: We are not including any 'label' properties because we decided to work with unlabeled nodes only.

In [1]:
from dataclasses import dataclass
from typing import List

@dataclass
class DatasetPoint:
    graph_1: List[List[int]] # List of edges which represents a graph, e.g: [[0, 8], [0, 5], [0, 2]...] 
    graph_2: List[List[int]] # List of edges which represents a graph, e.g: [[1, 3], [1, 5], [2, 4]...] 
    ged: int                 # The precise graph edit distance between graph_1 and graph_2, eg: 13

## Graph Manipulation

All the part related to manipulating graphs will be handled by [NetworkX](https://networkx.org/) library.

In [2]:
import networkx as nx

## WIP